<a href="https://colab.research.google.com/github/alyrazik/News_content_collect_store/blob/Crawler-that-follows-links/CrawlingBBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  News Content Collect and Store
## by Aly Abdelrazek
 
 


##Setup and Importing libraries

In [14]:
!python -m pip install pymongo[srv]

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle
import pymongo
from pymongo import MongoClient
from datetime import datetime
from time import sleep


In [2]:
pymongo.version

'3.11.0'

##Global variables

In [3]:
news_page_url = "https://bbc.com/news"
parent = "https://bbc.com"
SEARCH_LIMIT = 10   #limit the number of returned articles matching a keyword search


##Crawl


In [7]:
# Obtains news stories links from news_page_url (https://bbc.com/news)
def give_links(url, parent):
  ''' Take a url, and its parent address and returns a list of absolute addresses of web links orginitating from it.'''
  
  page_html = requests.get(url).text
  page_soup = BeautifulSoup(page_html, "html.parser")
  page_links = page_soup.findAll("a") #find all links

  #validate links
  output1 = [link.get("href") for link in page_links if link.get("href") is not None]
  output2 = [parent+link if link.startswith("/news") else link for link in output1]
  output3 = [link for link in output2 if "http" in link]


  return output3
 

def is_news_article(link):
  ''' 
  Take a web link address and tests whether the link contains a news article in BBC. Returns 1 if True, 0 otherwise.  
  ''' 
  if "/news" in link: #news articles have /news in address. consider not a news article if not.

    try:
      s = BeautifulSoup(requests.get(link).text , "html.parser")
      if s.find(class_= "css-16rg7hm-ContainerWithSidebarWrapper e1jl38b40") is not None: #this class is only available in news articles
        return 1
      else:
        return 0
        #sleep(0.1)
    except requests.exceptions.ConnectionError:
        print("Connection to page refused")
    except:
        print("An error occured while trying to connect to link")
    else:
        print("Parsing successful")
  return 0
#return news_urls.append([news_page_to_news_urls(other_urls[i], parent) for i in range(2,1,len(other_urls))])


In [12]:
news_urls = []
other_urls = [news_page_url]

for i in range(3):
  links = give_links(other_urls[i], parent)
  other_urls.pop(0) #no longer needed
  print("other urls 0 ", other_urls)
  print("links ", links)
  news_urls = news_urls + [link for link in links if (is_news_article(link))]
  print("news_urls 1", news_urls)
  other_urls = other_urls + [link for link in links if link not in news_urls]
  print("other_urls 1", other_urls)
  other_urls = list(dict.fromkeys(other_urls)) #remove duplicates from list.
  print("other_urls 2", other_urls)

other urls 0  []
links  ['https://www.bbc.co.uk', 'https://www.bbc.co.uk/accessibility/', 'https://account.bbc.com/account', 'https://www.bbc.co.uk', 'https://www.bbc.co.uk/news', 'https://www.bbc.co.uk/sport', 'https://www.bbc.co.uk/weather', 'https://www.bbc.co.uk/iplayer', 'https://www.bbc.co.uk/sounds', 'https://www.bbc.co.uk/cbbc', 'https://www.bbc.co.uk/cbeebies', 'https://www.bbc.co.uk/food', 'https://www.bbc.co.uk/bitesize', 'https://www.bbc.co.uk/arts', 'https://www.bbc.co.uk/taster', 'https://www.bbc.co.uk/news/localnews', 'https://www.bbc.co.uk/bbcthree', 'https://search.bbc.co.uk/search', 'https://bbc.com/news', 'https://bbc.com/news', 'https://bbc.com/news/election/us2020', 'https://bbc.com/news/coronavirus', 'https://bbc.com/news/av/10462520', 'https://bbc.com/news/world', 'https://bbc.com/news/world/us_and_canada', 'https://bbc.com/news/uk', 'https://bbc.com/news/business', 'https://bbc.com/news/technology', 'https://bbc.com/news/science_and_environment', 'https://bbc.co

In [13]:
len(news_urls)

74

In [14]:
len(other_urls)

112

## Scrape

In [ ]:
documents = []
document_index = 0

for url in news_urls:
    print("Fetching {}".format(url))
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")

    news_heading = (soup.find('h1')).text
    news_text = [p.text for p in soup.find('article').find_all('p')]
    news_time = (soup.find('time'))['datetime']

    documents.append({"Document_Index":str(document_index), "Heading":news_heading, "Article":news_text, "DateTime":news_time})
    document_index = document_index+1


##Saving to MongoDB

In [ ]:
#connect to client
client = MongoClient("mongodb+srv://aly:a@cluster0.4pfcp.mongodb.net/db?retryWrites=true&w=majority")

In [ ]:
#create a database
db = client["news_database"]

In [ ]:
#create a collection (a table)
bbc_news = db["bbc_news"]

In [ ]:
db['bbc_news'].delete_many({})

In [ ]:
bbc_news.insert_many(documents)

##Retrieving content from MongoDB

In [ ]:
def retrieve_documents(database, collection):
  ''' Take a MongoDB database and collection name, and returns all documents in collection to a pandas dataframe'''
  retrieved_documents = database[collection].find() #do not use database.collection, it is a string :)
  retrieved_df = pd.DataFrame(retrieved_documents)
  return retrieved_df

db = client["news_database"]
df = retrieve_documents(database = db , collection = 'bbc_news')

## Keyword search in article text

####Create a text index -required for text search

In [ ]:
db = client["news_database"]
db.bbc_news.create_index([
          ("Article", "text"),
          ("Heading", "text"),
  ],
  name="search_index",
  weights={
      'Article':25,
      'Heading':100
  }
)

'search_index'

#### Search function

In [ ]:
def search(search_text, database, collection):
  '''
  Take a a string containing keywords and outputs all relevant articles to any of them
  Args:
    -search_text: a string of keywords (<string>)
    -database: MongoDB database name (<pymongo.database.Database>)
    -collection: string with the name of MongoDB collection (<string>)
  Returns:
    -A pandas dataframe containing the MongoDB contents of the returned news articles
  '''
  returned_cursor = database[collection].find({"$text": {"$search": search_text}}).limit(SEARCH_LIMIT)
  df = pd.DataFrame(returned_cursor)
  return df


In [ ]:
output = search("france sdecular", db, 'bbc_news')

In [ ]:
output

,_id,Document_Index,Heading,Article,DateTime
0,5f9b3fa64f790f3d240e148c,28,France attack: Three killed in 'Islamist terro...,[.css-14iz86j-BoldText{font-weight:bold;}Three...,2020-10-29T21:34:43.000Z
1,5f9b3fa64f790f3d240e1473,3,France attack: Three killed in 'Islamist terro...,[.css-14iz86j-BoldText{font-weight:bold;}Three...,2020-10-29T21:34:43.000Z
2,5f9b3fa64f790f3d240e147c,12,"Covid: Merkel warns of 'long, hard winter' as ...",[.css-14iz86j-BoldText{font-weight:bold;}Germa...,2020-10-29T19:12:05.000Z
3,5f9b3fa64f790f3d240e147a,10,"Covid: Merkel warns of 'long, hard winter' as ...",[.css-14iz86j-BoldText{font-weight:bold;}Germa...,2020-10-29T19:12:05.000Z
4,5f9b3fa64f790f3d240e1479,9,US growth surged in summer but recovery is slo...,[.css-14iz86j-BoldText{font-weight:bold;}The U...,2020-10-29T14:28:58.000Z
